In [13]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database
conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select date, ticker, ret, roeq, mom12m
    from data
    where date>='2010-01' and date<='2017-12'
    order by date, ticker
    """, 
    conn
)
df = df.dropna()
df = df.set_index("date")

from pandas_datareader import DataReader as pdr

mkt = pdr(
  "F-F_Research_Data_Factors",
  "famafrench",
  start="2009-12",
  end="2017-12"
)

mkt = mkt[0] / 100
mkt.index = mkt.index.astype(str)
mkt["mkt"] = mkt["Mkt-RF"] + mkt["RF"]
mkt["lagmkt"] = mkt.mkt.shift()






In [2]:
df = df.join(mkt, how="left")

from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression

model = TransformedTargetRegressor(
    transformer=QuantileTransformer(
      output_distribution="normal"
    ),
    regressor=LinearRegression()
)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
  QuantileTransformer(output_distribution="normal"),
  PolynomialFeatures(degree=2),
  QuantileTransformer(output_distribution="normal"),
  model
)

X = df[["roeq", "mom12m", "lagmkt"]]
y = df.ret - df.mkt
pipe.fit(X, y)

,roeq,mom12m,lagmkt
0,-0.007854,0.850288,NaN
1,-0.025086,0.151564,NaN
2,-0.031166,0.992753,NaN
3,0.005932,0.142857,NaN
4,0.253323,-0.072072,NaN
